In [1]:
import pandas as pd

In [2]:
# Read the raw data from the csv file
df = pd.read_csv('../data/rice-yield-raw-data.csv', header=1)
df.head()

,Varieties,Ecotype,Province of release,Year of release,Yield (t/ha),Growth period (d),Height (cm),Pannicle length (cm),Panicle number (million/ha),Seed setting rate (%),1000-grain-weight (g),Grains per panicle,Seed length/width ratio,Filled grain number
0,shan you 2 hao,Indica hybrid,Guangdong,1978,6.0000,141.5,92.1,NaN,2.7975,81.4,28.05,131.9,NaN,107.3666
1,qing dong ai,Indica inbred,Guangdong,1978,6.0000,125.0,93.0,NaN,NaN,NaN,26.50,NaN,NaN,NaN
2,hong mei zao,Indica inbred,Guangdong,1978,5.2500,115.0,90.0,NaN,3.3750,92.5,28.50,87.5,NaN,80.9375
3,dong bai ai,Indica inbred,Guangdong,1978,6.0000,144.0,90.0,NaN,NaN,80.0,23.00,100.0,NaN,80.0000
4,he jiang 19 hao,Japonica inbred,Heilongjiang,1978,6.4425,NaN,85.0,14.0,NaN,NaN,26.50,NaN,NaN,NaN


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7686 entries, 0 to 7685
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Varieties                    7686 non-null   object 
 1   Ecotype                      7686 non-null   object 
 2   Province of release          7686 non-null   object 
 3   Year of release              7686 non-null   int64  
 4   Yield (t/ha)                 7686 non-null   float64
 5   Growth period (d)            7256 non-null   float64
 6   Height (cm)                  7376 non-null   float64
 7   Pannicle length (cm)         4999 non-null   float64
 8   Panicle number (million/ha)  5339 non-null   float64
 9   Seed setting rate (%)        6626 non-null   float64
 10  1000-grain-weight (g)        7479 non-null   float64
 11  Grains per panicle           4449 non-null   float64
 12  Seed length/width ratio      5114 non-null   float64
 13  Filled grain numbe

In [4]:
df['Ecotype'].value_counts()

Indica hybrid      4814
Japonica inbred    1809
Indica inbred       767
Japonica hybrid     296
Name: Ecotype, dtype: int64

In [5]:
# Select the features of interest
df_clean = df[['Ecotype', 'Panicle number (million/ha)', 'Growth period (d)', 'Yield (t/ha)']].copy()
df_clean.head()

,Ecotype,Panicle number (million/ha),Growth period (d),Yield (t/ha)
0,Indica hybrid,2.7975,141.5,6.0000
1,Indica inbred,NaN,125.0,6.0000
2,Indica inbred,3.3750,115.0,5.2500
3,Indica inbred,NaN,144.0,6.0000
4,Japonica inbred,NaN,NaN,6.4425


In [6]:
# Rename the features of interest
df_clean.rename(columns={'Yield (t/ha)': 'yield', 'Growth period (d)': 'growth', 'Panicle number (million/ha)': 'panicle'}, inplace=True)
df_clean.head()

,Ecotype,panicle,growth,yield
0,Indica hybrid,2.7975,141.5,6.0000
1,Indica inbred,NaN,125.0,6.0000
2,Indica inbred,3.3750,115.0,5.2500
3,Indica inbred,NaN,144.0,6.0000
4,Japonica inbred,NaN,NaN,6.4425


In [7]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7686 entries, 0 to 7685
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ecotype  7686 non-null   object 
 1   panicle  5339 non-null   float64
 2   growth   7256 non-null   float64
 3   yield    7686 non-null   float64
dtypes: float64(3), object(1)
memory usage: 240.3+ KB


In [8]:
# Remove rows with missing values
df_clean.dropna(axis=0, inplace=True)
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5285 entries, 0 to 7685
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Ecotype  5285 non-null   object 
 1   panicle  5285 non-null   float64
 2   growth   5285 non-null   float64
 3   yield    5285 non-null   float64
dtypes: float64(3), object(1)
memory usage: 206.4+ KB


In [9]:
df_clean['Ecotype'].value_counts()

Indica hybrid      3947
Indica inbred       582
Japonica inbred     571
Japonica hybrid     185
Name: Ecotype, dtype: int64

In [10]:
# Select the rows corresponding to the Japonica inbred type of rice plant 
df_clean = df_clean[df_clean['Ecotype'] == 'Japonica inbred']
df_clean['Ecotype'].value_counts()

Japonica inbred    571
Name: Ecotype, dtype: int64

In [11]:
df_clean.drop(columns=df_clean.columns[0], inplace=True)
df_clean.head()

,panicle,growth,yield
42,3.975,184.0,7.5000
71,3.975,184.0,6.7500
142,2.850,125.0,6.4275
183,4.500,147.5,7.2285
193,5.100,167.5,8.7150


Remove the outliers, with the 0.01 and 0.99 percentiles.

In [12]:
# Before removing outliers
df_clean.describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])

,panicle,growth,yield
count,571.000000,571.000000,571.000000
mean,3.498845,153.884238,8.871198
std,0.618679,15.913550,1.014709
min,2.023500,118.000000,4.687500
1%,2.541000,122.880000,6.395655
10%,2.865000,134.500000,7.596000
25%,3.060000,142.000000,8.380500
50%,3.345000,154.000000,8.823400
75%,3.802500,161.250000,9.456000
90%,4.408500,177.000000,10.185000


In [13]:
# Before removing outliers
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 571 entries, 42 to 7408
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   panicle  571 non-null    float64
 1   growth   571 non-null    float64
 2   yield    571 non-null    float64
dtypes: float64(3)
memory usage: 17.8 KB


In [14]:
# Remove values lower than or equal to the 0.01 percentile
df_clean = df_clean[df_clean['yield'] > df_clean['yield'].quantile(0.01)]
df_clean = df_clean[df_clean['growth'] > df_clean['growth'].quantile(0.01)]
df_clean = df_clean[df_clean['panicle'] > df_clean['panicle'].quantile(0.01)]

# Remove values higher than or equal to the 0.99 percentile
df_clean = df_clean[df_clean['yield'] < df_clean['yield'].quantile(0.99)]
df_clean = df_clean[df_clean['growth'] < df_clean['growth'].quantile(0.99)]
df_clean = df_clean[df_clean['panicle'] < df_clean['panicle'].quantile(0.99)]

In [15]:
# After removing outliers
df_clean.describe(percentiles=[0.01, 0.1, 0.25, 0.5, 0.75, 0.9, 0.99])

,panicle,growth,yield
count,534.000000,534.000000,534.000000
mean,3.470751,153.415169,8.887930
std,0.552669,14.840915,0.897845
min,2.550000,123.400000,6.401100
1%,2.625000,124.733000,6.490786
10%,2.895000,135.000000,7.713810
25%,3.063750,142.000000,8.403375
50%,3.307500,154.000000,8.855700
75%,3.750000,160.725000,9.419250
90%,4.296000,175.970000,10.099500


In [16]:
df_clean.head(6)

,panicle,growth,yield
42,3.975,184.0,7.5000
71,3.975,184.0,6.7500
142,2.850,125.0,6.4275
183,4.500,147.5,7.2285
193,5.100,167.5,8.7150
223,3.675,180.0,7.7820


In [17]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 534 entries, 42 to 7408
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   panicle  534 non-null    float64
 1   growth   534 non-null    float64
 2   yield    534 non-null    float64
dtypes: float64(3)
memory usage: 16.7 KB


In [18]:
df_clean.to_csv('../data/rice-yield-clean-data.csv', index=False)